In [1]:
import random
import numpy as np
import math
from difflib import SequenceMatcher

In [2]:
def generate_genome(length): #length = 10000
    bases = ['A', 'C', 'G', 'T']
    genome = ''.join(random.choices(bases, k=length))
    return genome

In [3]:
def generate_short_reads(genome, coverage_percentage, read_length=500):
    coverage = coverage_percentage / 100.0
    total_bases = len(genome)
    reads = []

    # Calculate the number of reads needed to achieve the desired coverage
    total_reads_needed = int(total_bases * coverage)
    reads_per_base = total_reads_needed / total_bases

    # Generate short reads based on the calculated number of reads per base
    current_base_index = 0
    while current_base_index < total_bases:
        # Calculate the number of reads to cover this base
        reads_for_this_base = max(1, int(reads_per_base))
        for _ in range(reads_for_this_base):
            read_end = min(current_base_index + read_length, total_bases)
            reads.append(genome[current_base_index:read_end])
        current_base_index += read_length

    return reads

In [4]:
def introduce_mutations(genome, mutation_rate):
    mutation_indices = random.sample(range(len(genome)), int(len(genome)*mutation_rate))
    reference_sequence = list(genome)
    for index in mutation_indices:
        reference_sequence[index] = random.choice(['A', 'C', 'G', 'T'])
    return ''.join(reference_sequence)

In [5]:
def count_mismatches(str1, str2):
    # Ensure both strings have the same length
    if len(str1) != len(str2):
        raise ValueError("Strings must have the same length")

    # Initialize a counter for mismatches
    mismatch_count = 0

    # Iterate through the characters of both strings simultaneously
    for char1, char2 in zip(str1, str2):
        # If characters are not equal, increment mismatch count
        if char1 != char2:
            mismatch_count += 1

    return mismatch_count

In [6]:
def trivial_mapping(full_read, short_reads, max_mismatches=3):
    new_full_read = list(' ' * len(full_read))  # Initialize with spaces
    for short_read in short_reads:
      matcher = SequenceMatcher(None, full_read, short_read)
      # print(matcher.get_matching_blocks())
      for i in range(len(matcher.get_matching_blocks())):
        if matcher.get_matching_blocks()[i][2]>1:
          if count_mismatches(full_read[matcher.get_matching_blocks()[0][0]:matcher.get_matching_blocks()[0][0]+len(short_read)], short_read) <  max_mismatches:
            new_full_read[matcher.get_matching_blocks()[0][0]:matcher.get_matching_blocks()[0][0]+len(short_read)]= short_read
            # print(new_full_read[matcher.get_matching_blocks()[0][0]:matcher.get_matching_blocks()[0][0]+len(short_read)])
    return ''.join(new_full_read)

In [7]:
genome = generate_genome(10000)

In [8]:
print(genome)

AAAAAGAACATCCAGTGCTTGCTTTTTGGCAGGTGCTCGACAGTTTTTGGTTAGGTGGAATGTTCGAATGTAAGACAGTGAAGGCATCTTGCGAAGCTGACCCTGAGAAGGAACCGTACGTTTCCATTAACTGCGAGCCACCTGCAAGAAGGCCTTCAAGCGCCTGGTTACAAAAATGCAATTATTGGCACCTCCCTGTCAATATGCTGCTATCTAAGGCCTCTAAATGAGTCGGGTTTATACTCATAGTTACATGTTGTCTAGGCAACATACTCGTAGTGAATTTGAGACAGCTGCGAACAAGCTGGGTCCCTCCTACCCGTTCGCCTGAGGCCGAGGCTCCACAAATCCCTACCCGTTCACGCTACAATACTTACCACGTTTGAAATGAGGTATCGCAGGGCATTACGCTGGGTAGGCACGGCGACTCCTTCTCGATAGCAGAGCGTATTCTGGCTCGCCGTTCTCTATGCGAGCCAGGTAGGTACTGAAAGTTACAATGATCCCCTGCAGTACCTATTGTAAAGTTGCTGAGATCCCACCCTATGAGATTTTTCGATGAGGCGGGAGGCTTGTCGTTGTGCCAGCTCCCCCCAGTATTTGGGCGCTTATCCCTCCCGTATAAGCGTGATTAAAGCCCTTGTTTACGTGTCTGTATGCTGATGATTCGTCACCCTCGAGATAAACGTGTCGTGGGTCGAGGAGCTGAGTAGATGTGTGCTGGATCTAATGGTTCTGCCATCGAATAATCATAGCGTTAGTAGGCTGGCACAGGATCCATTTACGGGTGGTCATGTGCTCGTTTAGATAACCTAACCCTGGCCATTACGATGTCACTCCACGAATCGTCTGGTTGGTTCCTTTCGAGGTCGGAACCCCCAAGCCACGAGAGCGCTCGATTTTCTGACAATGGGGCGACAGCTGACCGCCTGTTGTAAATTTAGACCACAAATGGTAGTGCGCGCGACTGAAGGATCGGGTGTCTCAGTGTCGAGACTTC

In [9]:
coverages = [100, 200, 300]  # Adjust coverages as needed
short_reads = [generate_short_reads(genome, coverage, read_length=50) for coverage in coverages]

In [10]:
print(len(short_reads),len(short_reads[0]),len(short_reads[0][0]), short_reads[0][0])

3 200 50 AAAAAGAACATCCAGTGCTTGCTTTTTGGCAGGTGCTCGACAGTTTTTGG


In [11]:
mutation_rate = random.uniform(0.05, 0.10)
reference_sequence = introduce_mutations(genome, mutation_rate)

In [12]:
if genome != reference_sequence:
    print("Reference sequence successfully generated.")
else:
    print("Reference sequence generation failed.")

Reference sequence successfully generated.


In [13]:
count_mismatches(genome, reference_sequence)

390

In [14]:
reconstructed_genome_mapping = trivial_mapping(reference_sequence,short_reads[0]) # RUNTIME = 12 seconds

In [15]:
print("genome: ",genome, "\nNew:    ", reconstructed_genome_mapping)

genome:  AAAAAGAACATCCAGTGCTTGCTTTTTGGCAGGTGCTCGACAGTTTTTGGTTAGGTGGAATGTTCGAATGTAAGACAGTGAAGGCATCTTGCGAAGCTGACCCTGAGAAGGAACCGTACGTTTCCATTAACTGCGAGCCACCTGCAAGAAGGCCTTCAAGCGCCTGGTTACAAAAATGCAATTATTGGCACCTCCCTGTCAATATGCTGCTATCTAAGGCCTCTAAATGAGTCGGGTTTATACTCATAGTTACATGTTGTCTAGGCAACATACTCGTAGTGAATTTGAGACAGCTGCGAACAAGCTGGGTCCCTCCTACCCGTTCGCCTGAGGCCGAGGCTCCACAAATCCCTACCCGTTCACGCTACAATACTTACCACGTTTGAAATGAGGTATCGCAGGGCATTACGCTGGGTAGGCACGGCGACTCCTTCTCGATAGCAGAGCGTATTCTGGCTCGCCGTTCTCTATGCGAGCCAGGTAGGTACTGAAAGTTACAATGATCCCCTGCAGTACCTATTGTAAAGTTGCTGAGATCCCACCCTATGAGATTTTTCGATGAGGCGGGAGGCTTGTCGTTGTGCCAGCTCCCCCCAGTATTTGGGCGCTTATCCCTCCCGTATAAGCGTGATTAAAGCCCTTGTTTACGTGTCTGTATGCTGATGATTCGTCACCCTCGAGATAAACGTGTCGTGGGTCGAGGAGCTGAGTAGATGTGTGCTGGATCTAATGGTTCTGCCATCGAATAATCATAGCGTTAGTAGGCTGGCACAGGATCCATTTACGGGTGGTCATGTGCTCGTTTAGATAACCTAACCCTGGCCATTACGATGTCACTCCACGAATCGTCTGGTTGGTTCCTTTCGAGGTCGGAACCCCCAAGCCACGAGAGCGCTCGATTTTCTGACAATGGGGCGACAGCTGACCGCCTGTTGTAAATTTAGACCACAAATGGTAGTGCGCGCGACTGAAGGATCGGGTGTCTCAGTGT

In [16]:
if genome == reconstructed_genome_mapping:
    print("Genome successfully reconstructed using trivial mapping.")
else:
    print("Genome reconstruction failed using trivial mapping.")


Genome reconstruction failed using trivial mapping.


In [17]:
reconstructed_genome_mapping = trivial_mapping(reference_sequence,short_reads[1]) #RUNTIME = 20 seconds

In [18]:
print("genome: ",genome, "\nNew:    ", reconstructed_genome_mapping)

genome:  AAAAAGAACATCCAGTGCTTGCTTTTTGGCAGGTGCTCGACAGTTTTTGGTTAGGTGGAATGTTCGAATGTAAGACAGTGAAGGCATCTTGCGAAGCTGACCCTGAGAAGGAACCGTACGTTTCCATTAACTGCGAGCCACCTGCAAGAAGGCCTTCAAGCGCCTGGTTACAAAAATGCAATTATTGGCACCTCCCTGTCAATATGCTGCTATCTAAGGCCTCTAAATGAGTCGGGTTTATACTCATAGTTACATGTTGTCTAGGCAACATACTCGTAGTGAATTTGAGACAGCTGCGAACAAGCTGGGTCCCTCCTACCCGTTCGCCTGAGGCCGAGGCTCCACAAATCCCTACCCGTTCACGCTACAATACTTACCACGTTTGAAATGAGGTATCGCAGGGCATTACGCTGGGTAGGCACGGCGACTCCTTCTCGATAGCAGAGCGTATTCTGGCTCGCCGTTCTCTATGCGAGCCAGGTAGGTACTGAAAGTTACAATGATCCCCTGCAGTACCTATTGTAAAGTTGCTGAGATCCCACCCTATGAGATTTTTCGATGAGGCGGGAGGCTTGTCGTTGTGCCAGCTCCCCCCAGTATTTGGGCGCTTATCCCTCCCGTATAAGCGTGATTAAAGCCCTTGTTTACGTGTCTGTATGCTGATGATTCGTCACCCTCGAGATAAACGTGTCGTGGGTCGAGGAGCTGAGTAGATGTGTGCTGGATCTAATGGTTCTGCCATCGAATAATCATAGCGTTAGTAGGCTGGCACAGGATCCATTTACGGGTGGTCATGTGCTCGTTTAGATAACCTAACCCTGGCCATTACGATGTCACTCCACGAATCGTCTGGTTGGTTCCTTTCGAGGTCGGAACCCCCAAGCCACGAGAGCGCTCGATTTTCTGACAATGGGGCGACAGCTGACCGCCTGTTGTAAATTTAGACCACAAATGGTAGTGCGCGCGACTGAAGGATCGGGTGTCTCAGTGT

In [19]:
if genome == reconstructed_genome_mapping:
    print("Genome successfully reconstructed using trivial mapping.")
else:
    print("Genome reconstruction failed using trivial mapping.")


Genome reconstruction failed using trivial mapping.


In [20]:
reconstructed_genome_mapping = trivial_mapping(reference_sequence,short_reads[2]) # Runtime = 28 seconds

In [21]:
print("genome: ",genome, "\nNew:    ", reconstructed_genome_mapping)

genome:  AAAAAGAACATCCAGTGCTTGCTTTTTGGCAGGTGCTCGACAGTTTTTGGTTAGGTGGAATGTTCGAATGTAAGACAGTGAAGGCATCTTGCGAAGCTGACCCTGAGAAGGAACCGTACGTTTCCATTAACTGCGAGCCACCTGCAAGAAGGCCTTCAAGCGCCTGGTTACAAAAATGCAATTATTGGCACCTCCCTGTCAATATGCTGCTATCTAAGGCCTCTAAATGAGTCGGGTTTATACTCATAGTTACATGTTGTCTAGGCAACATACTCGTAGTGAATTTGAGACAGCTGCGAACAAGCTGGGTCCCTCCTACCCGTTCGCCTGAGGCCGAGGCTCCACAAATCCCTACCCGTTCACGCTACAATACTTACCACGTTTGAAATGAGGTATCGCAGGGCATTACGCTGGGTAGGCACGGCGACTCCTTCTCGATAGCAGAGCGTATTCTGGCTCGCCGTTCTCTATGCGAGCCAGGTAGGTACTGAAAGTTACAATGATCCCCTGCAGTACCTATTGTAAAGTTGCTGAGATCCCACCCTATGAGATTTTTCGATGAGGCGGGAGGCTTGTCGTTGTGCCAGCTCCCCCCAGTATTTGGGCGCTTATCCCTCCCGTATAAGCGTGATTAAAGCCCTTGTTTACGTGTCTGTATGCTGATGATTCGTCACCCTCGAGATAAACGTGTCGTGGGTCGAGGAGCTGAGTAGATGTGTGCTGGATCTAATGGTTCTGCCATCGAATAATCATAGCGTTAGTAGGCTGGCACAGGATCCATTTACGGGTGGTCATGTGCTCGTTTAGATAACCTAACCCTGGCCATTACGATGTCACTCCACGAATCGTCTGGTTGGTTCCTTTCGAGGTCGGAACCCCCAAGCCACGAGAGCGCTCGATTTTCTGACAATGGGGCGACAGCTGACCGCCTGTTGTAAATTTAGACCACAAATGGTAGTGCGCGCGACTGAAGGATCGGGTGTCTCAGTGT

In [22]:
if genome == reconstructed_genome_mapping:
    print("Genome successfully reconstructed using trivial mapping.")
else:
    print("Genome reconstruction failed using trivial mapping.")


Genome reconstruction failed using trivial mapping.
